# Gradio Interface to Chat Description
This is the final product showcasing the CRM AI Assistant built in the Gradio environment. The current version uses DummyCRM, a test module that simulates the interface and functionality for demonstration purposes.

Our CRM AI Assistant can:

* Generate intelligent business insights and pipeline analyses using advanced prompts, providing evaluations of conversion rates, deal performance, and strategic recommendation for sales optimization (TO-DO).

* Produce detailed Matplotlib/Seaborn charts visualizing opportunity pipeline stages, revenue distribution by industry, lead conversion rates, and deal amount distributions across the sales organization.

Example queries it can handle are:
- 'Give me a summary of [account name]'
- 'Draft a follow-up email for [account name]'
- Just type any question to see what it can help with!

## Step 1: Import Packages

In [12]:
import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import time
import google.generativeai as genai
from dotenv import load_dotenv

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


## Step 2: Setup for Gemini and API Key

In [13]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [14]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')

Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'mo

## Step 3: Load Data

In [15]:
# Load clean data
base_path = "data_directory/clean_data"

# Read CSV files
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

## Step 4: Add Agent Classes

In [ ]:
# Account Summary Agent Class
class AccountSummaryAgent:
    def __init__(self, model_name="gemini-2.5-flash"):
        self.model = genai.GenerativeModel(model_name)
    
    def create_summary(self, account_data: dict, health_score: float = None):
        """Generate an account summary from account data."""
        
        # Build the prompt
        prompt = f"""
You are an expert CRM account analyst.

Generate a concise, executive-level account summary for:

**Account Data:**
{str(account_data)}

**Health Score:** {health_score if health_score else 'Not available'}

**Generate the following sections:**
1. Executive Summary (2-3 sentences)
2. Key Strengths (bullet points)
3. Key Risks (bullet points if any)
4. Recommended Next Actions (2-3 actions)
5. Priority Level (High/Medium/Low)

Be specific and data-driven.
"""
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating summary: {str(e)}"


# Email Drafting Agent Class
class EmailDraftingAgent:
    def __init__(self, model_name="gemini-2.5-flash"):
        self.model = genai.GenerativeModel(model_name)
    
    def draft_email(
        self,
        email_type: str,
        account_data: dict,
        opportunity_data: dict = None,
        tone: str = "professional"
    ):
        """Generate a sales email."""
        
        # Build the prompt
        prompt = f"""
You are an expert B2B sales communication specialist.

Generate a {email_type} email with a {tone} tone.

**Account Information:**
{str(account_data)}

**Opportunity Details:**
{str(opportunity_data) if opportunity_data else 'No opportunity data provided'}

**Requirements:**
- Include a compelling subject line
- Personalize based on account details
- Keep it concise (3-4 paragraphs)
- Include a clear call-to-action
- Professional closing

Generate the email now:
"""
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating email: {str(e)}"


print("Agent classes added")

### CRM and Search Modules

In [ ]:
class CRM:
    """
    CRM class that routes queries to appropriate agents.
    """
    
    def __init__(self, accounts_df, pipeline_df, teams_df, products_df):
        self.accounts = accounts_df
        self.pipeline = pipeline_df
        self.teams = teams_df
        self.products = products_df
        
        # Initialize both agents
        print("Initializing AI agents...")
        self.account_agent = AccountSummaryAgent()
        self.email_agent = EmailDraftingAgent()
        print("✓ AI agents ready")
    
    def process_query(self, query, context=None):
        """
        Process user queries and route to appropriate agent.
        
        Args:
            query (str): User's question
            context (dict): Optional context (account, agent, timeframe)
            
        Returns:
            str: Response from appropriate agent or direct answer
        """
        query_lower = query.lower()
        
        # Check for account summary requests
        if any(word in query_lower for word in ['summary', 'summarize', 'overview', 'tell me about']):
            return self._handle_account_summary(query, context)
        
        # Check for email drafting requests
        elif any(word in query_lower for word in ['email', 'draft', 'write', 'compose', 'send']):
            return self._handle_email_request(query, context)
        
        # Default: provide helpful guidance
        else:
            return self._handle_general_query(query, context)
    
    def _handle_account_summary(self, query, context):
        """Handle account summary requests."""
        try:
            # Try to get account from context
            account_name = None
            
            if context and 'account' in context:
                account_name = context['account']
            else:
                # Try to extract account name from query
                for acc in self.accounts['account'].unique():
                    if acc.lower() in query.lower():
                        account_name = acc
                        break
            
            if not account_name:
                return "Please specify an account name or select one from the filter."
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found."
            
            account_info = account_row.iloc[0]
            
            # Prepare account data
            account_data = {
                'account_name': account_info['account'],
                'sector': account_info['sector'],
                'revenue': account_info['revenue'],
                'employees': account_info['employees'],
                'location': account_info['office_location']
            }
            
            # Get opportunities for this account
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            if not account_opps.empty:
                won_deals = account_opps[account_opps['deal_stage'].str.lower() == 'won']
                account_data['total_opportunities'] = len(account_opps)
                account_data['won_deals'] = len(won_deals)
                account_data['total_revenue'] = won_deals['close_value'].sum()
            
            # Calculate simple health score
            health_score = 0.75  # Default
            if not account_opps.empty:
                won_count = len(won_deals)
                total_closed = len(account_opps[account_opps['deal_stage'].str.lower().isin(['won', 'lost'])])
                if total_closed > 0:
                    health_score = won_count / total_closed
            
            # Generate summary using agent
            print(f"Generating summary for {account_name}...")
            summary = self.account_agent.create_summary(account_data, health_score)
            
            return summary
            
        except Exception as e:
            return f"Error generating account summary: {str(e)}"
    
    def _handle_email_request(self, query, context):
        """Handle email drafting requests."""
        try:
            # Determine email type from query
            if 'follow' in query.lower():
                email_type = 'follow-up'
            elif 'intro' in query.lower():
                email_type = 'introduction'
            elif 'proposal' in query.lower():
                email_type = 'proposal'
            else:
                email_type = 'follow-up'  # Default
            
            # Get account from context
            account_name = context.get('account') if context else None
            
            if not account_name:
                return "Please select an account from the filter to draft an email."
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found."
            
            account_info = account_row.iloc[0]
            
            account_data = {
                'account_name': account_info['account'],
                'sector': account_info['sector'],
                'revenue': account_info['revenue'],
                'employees': account_info['employees']
            }
            
            # Get opportunity data if exists
            opportunity_data = None
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            
            if not account_opps.empty:
                opp = account_opps.iloc[0]
                opportunity_data = {
                    'product': opp['product'],
                    'deal_stage': opp['deal_stage'],
                    'close_value': opp['close_value']
                }
            
            # Determine tone
            tone = 'friendly' if 'friendly' in query.lower() else 'professional'
            
            # Generate email using agent
            print(f"Generating {email_type} email for {account_name}...")
            email = self.email_agent.draft_email(
                email_type=email_type,
                account_data=account_data,
                opportunity_data=opportunity_data,
                tone=tone
            )
            
            return email
            
        except Exception as e:
            return f"Error generating email: {str(e)}"
    
    def _handle_general_query(self, query, context):
        """Handle general queries with helpful guidance."""
        
        # Provide some quick stats
        response = "**CRM AI Assistant - What I Can Help With:**\n\n"
        response += "📊 **Account Summaries**: Ask me to summarize any account\n"
        response += "   Example: 'Give me a summary of [account name]'\n\n"
        response += "✉️ **Email Drafting**: I can draft sales emails for you\n"
        response += "   Example: 'Draft a follow-up email for [account name]'\n\n"
        response += "📈 **Quick Stats**:\n"
        response += f"   • Total Accounts: {len(self.accounts)}\n"
        response += f"   • Total Opportunities: {len(self.pipeline)}\n"
        response += f"   • Won Deals: {len(self.pipeline[self.pipeline['deal_stage'].str.lower() == 'won'])}\n\n"
        response += "💡 **Tip**: Select an account from the filter and ask me about it!"
        
        return response


In [ ]:
# Create the CRM instance
crm = CRM(
    accounts_df=accounts,
    pipeline_df=pipeline,
    teams_df=teams,
    products_df=products
)

### Data Visualization & Analytics

In [27]:
# Task 3c: Data Visualization & Analytics (Starter)
def generate_chart(account=None, agent=None, timeframe=None):
    fig, ax = plt.subplots(figsize=(10, 11))
    df = pipeline.copy()

    if account:
        df = df[df["account"] == account]
    if agent:
        df = df[df["sales_agent"] == agent]
    if timeframe:
        df = df[df["engage_date"] >= timeframe]

    summary = df.groupby("deal_stage")["close_value"].sum()
    summary.plot(kind="bar", ax=ax)
    ax.set_title("Deal Value by Stage")
    ax.set_ylabel("Close Value ($)")
    ax.set_xlabel("Deal Stage")

    return fig

### Chatbot Implementation

In [22]:
# CRM chatbot function
def crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter):
    context = {}
    if account_filter and account_filter != "None":
        row = accounts[accounts["account"] == account_filter].iloc[0].to_dict()
        context.update(row)
    if agent_filter and agent_filter != "None":
        context["sales_agent"] = agent_filter
    if timeframe_filter:
        context["timeframe"] = timeframe_filter

    llm_response = crm.process_query(user_query, context=context)
    chart = generate_chart(
        account_filter if account_filter != "None" else None,
        agent_filter if agent_filter != "None" else None,
        timeframe_filter
    )
    return llm_response, chart

In [23]:
# wrapper to handle "None" dropdowns
account_choices = ["None"] + sorted(accounts["account"].unique())
agent_choices = ["None"] + sorted(pipeline["sales_agent"].unique())

def crm_chatbot_wrapper(user_query, account_filter, agent_filter, timeframe_filter):
    if account_filter == "None":
        account_filter = None
    if agent_filter == "None":
        agent_filter = None
    return crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter)

### Final Gradio Interface

In [30]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center; font-size: 40px;'>CRM AI Assistant</h1>")
    gr.Markdown("<h2 style='text-align: center; font-size: 20px;'>Salesforce Team2B</h2>")
    gr.Markdown("<p style='text-align: center; font-size: 15px;'>Our CRM AI Assistant is your go-to buddy for valuable insights. It’ll help you get any answer you need!</p>")
    gr.Markdown("<p style='text-align: center; font-size: 16px; font-weight: bold;'>Ask Questions. Get Insights. See Charts.</p>")

    with gr.Row():
        user_query = gr.Textbox(label="Your Question")
    
    with gr.Row():
        account_filter = gr.Dropdown(
            choices=account_choices,
            label="Filter by Account (optional)",
            value="None"
        )
        agent_filter = gr.Dropdown(
            choices=agent_choices,
            label="Filter by Sales Agent (optional)",
            value="None"
        )
        timeframe_filter = gr.Textbox(
            label="Filter by Date (YYYY-MM-DD)",
            placeholder="2025-01-01"
        )

    with gr.Row():
        submit_btn = gr.Button("Generate Insight", elem_id="submit-btn")

    gr.HTML("""
    <style>
    #submit-btn {
        background-color: #1E90FF; /* Dodger Blue */
        color: white;
        font-weight: bold;
        border-radius: 8px;
        padding: 10px 20px;
        font-size: 16px;
    }
    #submit-btn:hover {
        background-color: #104E8B; /* Darker blue on hover */
    }
    </style>
    """)

    with gr.Row():
        llm_output = gr.Textbox(label="Insight", lines=10)
    
    chart_output = gr.Plot(label="Data Visualization")

    submit_btn.click(
        crm_chatbot_wrapper,
        inputs=[user_query, account_filter, agent_filter, timeframe_filter],
        outputs=[llm_output, chart_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7929
* To create a public link, set `share=True` in `launch()`.
